In [1]:
!pip install pyspark >> None

In [16]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import avg, col

In [4]:
spark = SparkSession.builder.appName("TestDataFrame").getOrCreate()

In [5]:
data = [("2023-11-20", "Electronics", 100, 12000),
         ("2023-11-21", "Electronics", 110, 13000),
         ("2023-11-22", "Electronics", 105, 12500),
         ("2023-11-20", "Clothing", 300, 15000),
         ("2023-11-21", "Clothing", 280, 14000),
         ("2023-11-22", "Clothing", 320, 16000),
         ("2023-11-20", "Books", 150, 9000),
         ("2023-11-21", "Books", 200, 12000),
         ("2023-11-22", "Books", 180, 10000)]

In [6]:
columns = ["date", "category", "quantity", "revenue"]
df = spark.createDataFrame(data, columns)

In [7]:
df.show()

+----------+-----------+--------+-------+
|      date|   category|quantity|revenue|
+----------+-----------+--------+-------+
|2023-11-20|Electronics|     100|  12000|
|2023-11-21|Electronics|     110|  13000|
|2023-11-22|Electronics|     105|  12500|
|2023-11-20|   Clothing|     300|  15000|
|2023-11-21|   Clothing|     280|  14000|
|2023-11-22|   Clothing|     320|  16000|
|2023-11-20|      Books|     150|   9000|
|2023-11-21|      Books|     200|  12000|
|2023-11-22|      Books|     180|  10000|
+----------+-----------+--------+-------+



In [9]:
window = Window.partitionBy("category")

In [10]:
avg_revenue = df.withColumn("avg_revenue", F.avg("revenue").over(window))

In [11]:
avg_revenue.select("category", "revenue", "avg_revenue").show()

+-----------+-------+------------------+
|   category|revenue|       avg_revenue|
+-----------+-------+------------------+
|      Books|   9000|10333.333333333334|
|      Books|  12000|10333.333333333334|
|      Books|  10000|10333.333333333334|
|   Clothing|  15000|           15000.0|
|   Clothing|  14000|           15000.0|
|   Clothing|  16000|           15000.0|
|Electronics|  12000|           12500.0|
|Electronics|  13000|           12500.0|
|Electronics|  12500|           12500.0|
+-----------+-------+------------------+



In [19]:
avg_revenue_by_date = df.withColumn("avg_revenue", avg(col("revenue")).over(window))

In [20]:
pivoted_df = avg_revenue_by_date.groupBy("category").pivot("date").agg(F.first("avg_revenue"))

In [21]:
pivoted_df.show()

+-----------+------------------+------------------+------------------+
|   category|        2023-11-20|        2023-11-21|        2023-11-22|
+-----------+------------------+------------------+------------------+
|      Books|10333.333333333334|10333.333333333334|10333.333333333334|
|   Clothing|           15000.0|           15000.0|           15000.0|
|Electronics|           12500.0|           12500.0|           12500.0|
+-----------+------------------+------------------+------------------+

